# Two-Level Landau-Zener System

In this notebook we compare analytical results derived for a two-level Landau-Zener (LZ) system to those generated by a SchWARMA-driven numerical simulation. Analytical results for the two-level LZ model in the presence of colored noise were derived in Ref. [1]. We will show that numerical simulation of the LZ dynamics when driven by a SchWARMA model is in good agreement with analytical results.

The faulty two-level LZ system is described by

$$
H(t) = H_{LZ}(t) + H_{err}(t)
$$

where $H_{LZ}(t) = \Theta(t)\cdot \vec{S}$ captures the ideal dynamics and $H_{err}(t) = \vec{\beta}(t)\cdot \vec{S}$ represents the error Hamiltonian. $\vec{S}$ is the total spin vector involving all three generators of $SU(2)$. We will focus our attention on a specific case of this faulty system. Namely, $\Theta(t)=(2\Delta, 0, 2\alpha t)$, where $\alpha$ is the constant sweep velocity, $\Delta$ is the tunneling coupling matrix element, and $t$ represents a linear ramping that will take place from $t=-\infty$ to $t=\infty$. In addition, we will focus on the specific case of $\vec{\beta}(t)=(\beta_x(t),0,0)$, where $\beta_x(t)$ is a Gaussian random variable with zero mean and correlation function $C(t-t')=\langle\beta_x(t)\beta_x(t')\rangle$.

In [ ]:
import numpy as np
import mezze
import mezze.random.SchWARMA as schwarma
import aqc_implementations as aqc_imps
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import mpmath as mp
import math
import scipy.linalg as la

## Analytical Functions

The analytical formulas for transition probability to which we will compare SchWARMA simulation are described by Eq. (3.26) in Ref. [1]. In the case of slow noise, the transition probability is given by

$$
P_x(t) = 1 - \frac{\exp(-2\pi\lambda \Phi_1(t))}{\sqrt{1+\frac{2\pi\eta^2}{\alpha}\left[F(t) + \ln W(t)\right]}},
$$

where $\lambda=\Delta^2/2\alpha$ and

$$
\Phi_1(t) = \frac{F(t) + \ln W(t)}{1+\frac{2\pi\eta^2}{\alpha}\left[F(t) + \ln W(t)\right]}.
$$

The function $F(t)$ is defined as

$$
F(t) = \frac{1}{2}\left\{\left[ c\left(\sqrt{\frac{2\alpha}{\pi}}t\right) + \frac{1}{2}\right]^2 + \left[s\left(\sqrt{\frac{2\alpha}{\pi}}t\right) + \frac{1}{2}\right]^2\right\},
$$

where $c(t)$ and $s(t)$ are defined as the cosine and sine Fresnel integrals, respectively. The remaining function $\ln W(t)$ is given by

$$
\ln W(t) = -\frac{1}{2\pi\lambda}\ln\left\{1-G(t,\lambda)\exp\left[2\pi\lambda F(t)\right]\right\}
$$

with 

$$
G(t,\lambda) = \lambda e^{-\pi\lambda/2}|D_{-i\lambda-1}(-iz)|^2-\left(1-e^{-2\pi\lambda F(t)}\right).
$$

$D_n(x)$ is the parabolic cylinder (Weber's) function and $z=\sqrt{2\alpha}te^{-i\pi/4}$.

In [ ]:
def F(alpha, t):
    s,c = sp.fresnel(np.sqrt(2*alpha/math.pi)*t)
    f = 0.5*( (c + 0.5)**2 + (s + 0.5)**2 )
    return f

def G(alpha, lamb, t):
    z = math.sqrt(2*alpha)*t*np.exp(-1.0j*np.pi/4)
    d = mp.pcfd(-1.0j*lamb-1, -1.0j*z)
    g = lamb*np.exp(-np.pi*lamb/2)*abs(d)**2 - (1 - np.exp(-2*np.pi*lamb*F(alpha,t)))
    return g

def lnW(alpha, lamb, t):
    #print(1 - G(alpha, lamb, t)*math.exp(2*np.pi*lamb*F(alpha,t)))
    w = -1/(2*math.pi*lamb)*math.log(1 - G(alpha, lamb, t)*math.exp(2*np.pi*lamb*F(alpha,t)))
    return w

def phi(alpha, lamb, eta, t):
    kappa = F(alpha, t) + lnW(alpha, lamb, t)
    return kappa/(1+2*np.pi*eta**2/alpha*kappa)

def Psn_x(alpha, lamb, eta, t): 
    kappa = F(alpha,t) + lnW(alpha,lamb,t)
    px = 1 - np.exp(-2*np.pi*lamb*phi(alpha,lamb,eta,t))/np.sqrt(1 + 2*np.pi*eta**2/alpha*kappa)
    return px

We will now consider a particular set of parameters to convey the analytical formulas. These results will then be compared to the SchWARMA simulation under the same parameter choices.

In [ ]:
# Ideal Hamiltonian Parameters
lamb = 0.25 # Manuscript also consider lamb = 0.1, 1.0
delta = 1.0
alpha = delta**2/(2*lamb)

# Error Hamiltonian parameters
gamma0 = 0.01
gamma = gamma0*np.sqrt(alpha)
nu = 0.1
eta = np.sqrt(alpha*nu/np.pi)

# Data Collection for varying t
t_list = np.linspace(-20, 20, 1000)*1/np.sqrt(alpha)
analytical_fid_list = []
for t in t_list:
    analytical_fid_list.append([t*np.sqrt(alpha), Psn_x(alpha, lamb, eta,t)])
analytical_fid_list = np.array(analytical_fid_list)

In [ ]:
fig = plt.figure(dpi=70)
ax = fig.add_subplot(111)
ax.plot(analytical_fid_list[:,0], analytical_fid_list[:,1])
ax.set_xlim([-20,20])
ax.set_ylim([0,1.0])
ax.set_xlabel(r'$t\sqrt{\alpha}$', fontsize=20)
ax.set_ylabel(r'$P_x(t)$', fontsize=20);

## SchWARMA Simulation

We effectively simulate $H_E(t)$ and the faulty LZ dynamics by introducing a SchWARMA model into the Trotterized evolution as

$$
    U(T)=\exp\left(-i\int^T_0 H(t)dt\right) \approx \prod^{N}_{j=1} U_E(t_j)U_{ad}(t_j, t_{j-1}),
$$

where $U_E(t_j)=\exp(-i \sum_k y^k_{j} S_k)$. The variables $y^k_{j}$ are defined by the underlying SchWARMA model and thus, they encapsulate the temporal properties of the noise. The evolution

$$
U_{LZ}(t_j, t_{j-1}) = \exp\left(-i\int^{t_j}_{t_{j-1}} H_{LZ}(t)dt\right)\approx\prod^{N_j}_{k=1}e^{-iH_{LZ}(k\delta t)\delta t}
$$

represents the pure LZ dynamics that is Trotterized a number of steps $N_j=T/\delta t$, where $\delta t$ is the time resolution of the approximate evolution. Simulating the dynamics in this manner allows one to leverage the noiseless approximation of the LZ dynamics as a base for the faulty simulations.

We consider the noise correlation function $C(t-t')=\eta^2\sqrt{\pi} e^{-(t-t')^2/4\tau_c}$, where $\tau_c$ is the correlation time. Note that while the original results are derived for a Lorentzian spectrum, (1) they can be quickly extended to a Gaussian correlation function and (2) in the limit of $\tau_c \gg 0$, both spectrums should produce relatively similar results; we show this is indeed the case below.

In [ ]:
def run_ideal_lzs_model(config_specs, controls):    
    # Simulation configuration
    config = mezze.SimulationConfig() # Get the config structure (class)
    config.num_steps = config_specs['NN_mezze'] # Number of time steps to use in the simulation (larger numbers are more accurate)
    config.time_length = config_specs['T'] # Set the time length of the integration
    config.dt = float(config.time_length) / float(config.num_steps) # Set the time-step size
    config.parallel = False # Set to true for parallel Monte Carlo, False otherwise
    config.num_runs = config_specs['num_runs'] # This sets the number of Monte Carlo trials to use to compute the average superoperator
    config.get_unitary = True # Report unitary as opposed to Liouvillian
    config.time_sampling = True # set to true to get the channel at intermediate times
    config.sampling_interval = config_specs['record_steps'] # records the channel after every N time-steps
    config.realization_sampling = True # set true if you want to record every noise (monte-carlo) realization
    
    # Instantiate the pmd object
    pmd = aqc_imps.LZSTransverse()
    # pmd.noise_hints[0]['type'] = 'ArbitrarySpectrum'
    # pmd.noise_hints[0]['spectrum'] = spectrum
    pmd.noise_hints[0]['type'] = 'gauss'
    pmd.noise_hints[0]['amp'] = 0
    pmd.noise_hints[0]['corr_time'] = 1
    
    ## Run simulation
    ham_func = mezze.HamiltonianFunction(pmd)
    ham = mezze.PrecomputedHamiltonian(pmd, controls, config, ham_func.get_function())
    sim = mezze.Simulation(config, pmd, ham)
    report = sim.run()
    return report

In [ ]:
# Ideal Hamiltonian Parameters
lamb = 0.25 # Manuscript also consider lamb = 0.1, 1.0
delta = 1.0
alpha = delta**2/(2*lamb)
T = 600/np.sqrt(alpha)

# Error Hamiltonian parameters
gamma0 = 0.01
gamma = gamma0*np.sqrt(alpha)
nu = 0.1
eta = np.sqrt(alpha*nu/np.pi)

# SchWARMA parameters
a = [1,]
b = schwarma.gaussian_ma(1/gamma)
schwarma_every_n = 10
schwarma_realizations = 100

# Simulation config specs
config_specs = {'T':T, 'NN_mezze': 24000, 'num_runs': 1, 'record_steps': schwarma_every_n, 'sh_realz': schwarma_realizations}

# Control
x_ramp = 2*delta*np.ones(config_specs['NN_mezze'])
z_ramp = 2*alpha*np.linspace(-1,1,config_specs['NN_mezze'])*T/2
controls = np.transpose([x_ramp, z_ramp])

# Run Ideal Simulation
report = run_ideal_lzs_model(config_specs, controls)

# Run Faulty Simulation Using SchWARMA
NN_specs = int(config_specs['NN_mezze']//schwarma_every_n)
sx = np.array([[0,1],[1,0]])
rho0 = np.array([[1,0],[0,0]], dtype=complex)
rho11_snapshots = np.zeros([config_specs['sh_realz'], NN_specs])
for realz in range(config_specs['sh_realz']):
    print(realz)
    schwarma_angles = schwarma.schwarma_trajectory(NN_specs, a, b, amp=eta**2*np.sqrt(np.pi))
    U = np.array([[1,0],[0,1]], dtype=complex)
    U_prev = np.array([[1,0],[0,1]], dtype=complex)
    for i in range(NN_specs):
        Uk = report.time_samples[0][i].liouvillian()
        Ut = Uk @ np.conjugate(np.transpose(U_prev))
        Un = la.expm(-1.0j*schwarma_angles[i]/2*sx)
        U_prev = Uk
        U = Un @ Ut @ U
        rhot = U @ rho0 @ np.conjugate(np.transpose(U))
        rho11_snapshots[realz,i] = np.real(rhot[1,1])

We present the final numerical data as the mean and 95% confidence interval taken over the set of SchWARMA realizations.

In [ ]:
def jackknife_mean(data):
    mean_data = []
    for i in range(len(data)):
        temp_data = list(data[:i]) + list(data[(i+1):])
        mean_data.append(np.mean(temp_data))
    return np.mean(mean_data)

def jackknife_mean_CI(data):
    mean_data = []
    for i in range(len(data)):
        temp_data = list(data[:i]) + list(data[(i+1):])
        mean_data.append(np.mean(temp_data))
    return np.quantile(mean_data, 0.0257), np.quantile(mean_data, 0.975)

In [ ]:
rho11_mean = np.array([(jackknife_mean(rho11_snapshots[:,i]),jackknife_mean_CI(rho11_snapshots[:,i])) for i in range(NN_specs)])

In [ ]:
time_list = np.linspace(-T/2, T/2, NN_specs)
mean = rho11_mean[:,0]
CIs = rho11_mean[:,1]
yerr = [[mean[i]-CIs[i][0], CIs[i][1]-mean[i]] for i in range(len(mean))]

fig = plt.figure(dpi=70)
ax = fig.add_subplot(111)
ax.plot(analytical_fid_list[:,0], analytical_fid_list[:,1], label='Analytical')
ax.errorbar(time_list*np.sqrt(alpha), rho11_mean[:,0], yerr=np.transpose(yerr), fmt='o', markerfacecolor="None", label='SchWARMA')
ax.set_xlim([-20,20])
ax.set_ylim([0,1.0])
ax.set_xlabel(r'$t\sqrt{\alpha}$', fontsize=20)
ax.set_ylabel(r'$P_x(t)$', fontsize=20)
ax.legend(loc=2, fontsize=15)

## References

[1] M.B. Kenmoe, H.N. Phien, M.N. Kiselev, and L.C. Fai, Phys. Rev. B 87, 224301 (2013)